In [2]:
import os
import random
import numpy as np
import librosa.display
import soundfile as sf
import json as json
import pandas as pd
import torch

In [3]:
def get_files_count(folder_path):
	dirListing = os.listdir(folder_path)
	return len(dirListing)

In [5]:
import multiprocessing as mp
import tqdm
mp.cpu_count()

16

In [6]:
'''
PATH INITIALIZATION

PATH = Origninal Audio Files
SAVE_PATH = Transformed Audio Files
META_DATA_PATH = Metadata.json location
'''
import os
PATH = "/Users/valleotb/Desktop/Valleotb/sample"
SAVE_PATH = "/Users/valleotb/Desktop/Valleotb/sample_save_2"
META_DATA_PATH = '/Users/valleotb/Desktop/Valleotb/sample_metadata'
FILE_COUNT = get_files_count(PATH)
# FILE_LIST = os.listdir(PATH)

'''
NAMING SETTING
'''
j = 100000  # 원본 파일 네이밍
z = 200000  # 수정 후 파일 네이밍
kiki = 0
'''
PARAMETER SETTING
'''
SAMPLE_TIME = 16000 * 3 # (16000 * second)
OUTPUT_DIMENSION = 10

'''
META DATA INITIALIZATION
'''
META_DATA = f"{META_DATA_PATH}/metadata.json"
DATA = {}
# DATA['speech_segments'] = []

for i in range(FILE_COUNT - 1):
# for fname in FILE_LIST:
    j += 1
    '''
    AUDIO PREPROCESSING (CUT TO SAME TIMES)
    '''

    if z % 1000 == 0 :
        kiki += 1000
        os.mkdir(f"{SAVE_PATH}/{kiki}")
        FILE_BATCH_DIR = f"{SAVE_PATH}/{kiki}"
        print(f'complete{z}')

    WAVEFORM, SR = sf.read(f"{PATH}/{j}.flac")
    WAVEFORM = librosa.util.normalize(WAVEFORM)
    NON_SILENT_TIME = librosa.effects.split(WAVEFORM, top_db=20)

    WAVEFORM = WAVEFORM[NON_SILENT_TIME[0][0] : (NON_SILENT_TIME[0][0]+SAMPLE_TIME)]
    NON_SILENT_TIME_2 = librosa.effects.split(WAVEFORM, top_db=15)
    # print(NON_SILENT_TIME_2)

    if len(NON_SILENT_TIME_2) > 2 :
        random_num = random.randint(0, len(NON_SILENT_TIME_2)-1)
        k = NON_SILENT_TIME_2[random_num]
        if (k[1] - k[0]) > (SAMPLE_TIME / OUTPUT_DIMENSION) :

            '''
            DATA TRANSFORM PROCESS
            '''
            # Pitch Shift Process
            N_STEPS = random.randint(1,5)
            WAVEFORM_TRANSFORM = librosa.effects.pitch_shift(WAVEFORM[k[0]:k[1]], sr=SR, n_steps=N_STEPS)
            WAVEFORM_TRANSFORM_MINUS = librosa.effects.pitch_shift(WAVEFORM_TRANSFORM, sr=SR, n_steps=-N_STEPS)

            # Normalize
            WAVEFORM_NOT_TRANSFORM_PART = WAVEFORM[k[0]:k[1]]
            WAVEFORM_DOUBLED = WAVEFORM_NOT_TRANSFORM_PART ** 2
            WAVEFORM_TRANSFORM_DOUBLED = WAVEFORM_TRANSFORM_MINUS ** 2
            a = np.sqrt((WAVEFORM_DOUBLED.sum()) / (WAVEFORM_TRANSFORM_DOUBLED.sum()))

            WAVEFORM_TRANSFORM_FINAL = WAVEFORM_TRANSFORM_MINUS * a
            WAVEFORM[k[0]:k[1]] = WAVEFORM_TRANSFORM_FINAL

            '''
            DATA LABELING PROCESS

            LABELING => 구간의 50% 이상이 포함되었을때만 Labeling 진행한다.
            LABEL_DATA, POINTER -> Initializing
            '''
            LABEL_DATA = []
            POINTER = 0

            sf.write(f'{FILE_BATCH_DIR}/{z}.wav', WAVEFORM, SR, subtype="PCM_24")
            # save_fname = os.path.join(SAVE_PATH, fname[:-4] + '.wav')
            # sf.write(save_fname, WAVEFORM, SR, subtype="PCM_24")
            for e in range(OUTPUT_DIMENSION):
                if POINTER + (SAMPLE_TIME / OUTPUT_DIMENSION) <= k[0]:
                    # save label
                    LABEL_DATA.append(0)
                    POINTER += (SAMPLE_TIME / OUTPUT_DIMENSION)     # POINTER += 4800

                elif k[0] < POINTER + (SAMPLE_TIME / OUTPUT_DIMENSION) <= k[1]:
                    # save label
                    LABEL_DATA.append(1)
                    POINTER += (SAMPLE_TIME / OUTPUT_DIMENSION)     # POINTER += 4800

                elif POINTER + (SAMPLE_TIME / OUTPUT_DIMENSION) > k[1]:
                    # save label
                    LABEL_DATA.append(0)
                    POINTER += (SAMPLE_TIME / OUTPUT_DIMENSION)     # POINTER += 4800

            # write metadata
            DATA[f"/content/drive/MyDrive/PSAD/sample_save/{kiki}/{z}.wav"] = LABEL_DATA

            # process end

            z += 1

with open(META_DATA, 'w') as outfile:
    json.dump(DATA, outfile)

complete200000
complete201000
complete201000
complete202000
complete203000
complete203000
complete204000
complete204000
complete204000
complete204000
complete205000
complete205000
complete206000
complete206000
complete207000
complete207000
complete207000
complete207000
complete208000
complete209000
complete209000
complete210000
complete211000
complete212000
complete213000
complete214000
complete215000
complete215000
complete216000
complete217000
complete218000
complete218000
complete218000
complete219000
complete219000
complete220000
complete220000
complete220000
complete220000
complete221000
complete221000
complete221000
complete222000
complete222000
complete223000
complete223000
complete223000
complete224000
complete225000
complete226000
complete226000
complete226000
complete227000
complete227000
complete227000
complete228000
complete229000
complete230000
complete231000
complete232000
complete233000
complete234000
complete235000
complete235000
complete236000
complete237000
complete23

In [9]:
preprocess_single_data(os.path.join(PATH, "196876.flac"))

[[    0  7680]
 [11776 13824]
 [16384 23040]
 [27648 31744]
 [33792 35328]
 [38400 40448]
 [43008 48000]]


<module 'multiprocessing' from '/Users/valleotb/anaconda3/lib/python3.9/multiprocessing/__init__.py'>

In [11]:
with mp.Pool(processes=4) as pool:
    metadata_list = list(
        tqdm.tqdm(
            pool.imap_unordered(
                preprocess_single_data, FILE_LIST[:10],
            ),
            total=len(FILE_LIST),
            desc=f'preprocessing in progress'
        )
    )
len(metadata_list)
with open(META_DATA, 'w') as outfile:
    json.dump(DATA, outfile)

preprocessing in progress:   0%|          | 0/97629 [00:00<?, ?it/s]Process SpawnPoolWorker-694:
Traceback (most recent call last):
  File "/Users/valleotb/anaconda3/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/Users/valleotb/anaconda3/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/valleotb/anaconda3/lib/python3.9/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/Users/valleotb/anaconda3/lib/python3.9/multiprocessing/queues.py", line 368, in get
    return _ForkingPickler.loads(res)
AttributeError: Can't get attribute 'preprocess_single_data' on <module '__main__' (built-in)>
Process SpawnPoolWorker-695:
Traceback (most recent call last):
  File "/Users/valleotb/anaconda3/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/Users/valleotb/anaconda3/lib/python3.9/multiprocessing/process.py", line 108, in run
 

KeyboardInterrupt: 

In [53]:
# meta_data_json = pd.read_json('/Users/valleotb/Desktop/Valleotb/sample_metadata/metadata.json')
meta_data_json = json.load(open('/Users/valleotb/Desktop/Valleotb/sample_metadata/metadata.json'))
print(meta_data_json)

{'200000.wav': [1, 1, 0, 0, 0, 0, 0, 0, 0, 0], '200001.wav': [1, 1, 1, 1, 1, 0, 0, 0, 0, 0], '200002.wav': [0, 1, 0, 0, 0, 0, 0, 0, 0, 0], '200003.wav': [0, 0, 0, 0, 0, 0, 0, 1, 1, 1], '200004.wav': [1, 1, 1, 1, 1, 1, 0, 0, 0, 0], '200005.wav': [0, 0, 0, 1, 1, 0, 0, 0, 0, 0], '200006.wav': [1, 1, 0, 0, 0, 0, 0, 0, 0, 0], '200007.wav': [0, 0, 0, 0, 0, 0, 0, 1, 0, 0], '200008.wav': [0, 0, 0, 0, 0, 0, 1, 0, 0, 0], '200009.wav': [0, 0, 0, 0, 0, 1, 1, 1, 0, 0], '200010.wav': [0, 0, 0, 0, 0, 0, 0, 0, 1, 1], '200011.wav': [0, 0, 0, 0, 0, 0, 0, 0, 1, 0], '200012.wav': [0, 0, 0, 0, 1, 1, 1, 0, 0, 0], '200013.wav': [0, 0, 0, 0, 1, 1, 1, 0, 0, 0]}


In [56]:
m = list(meta_data_json.keys())
m[0]

'200000.wav'

In [62]:
meta_data_json[m[0]]

[1, 1, 0, 0, 0, 0, 0, 0, 0, 0]

In [63]:
print(m)

['200000.wav', '200001.wav', '200002.wav', '200003.wav', '200004.wav', '200005.wav', '200006.wav', '200007.wav', '200008.wav', '200009.wav', '200010.wav', '200011.wav', '200012.wav', '200013.wav']


In [64]:
print(len(meta_data_json))

14


In [11]:
import torch
tuio = torch.tensor([[1,2],[0,3],[4,5]])
tuio = torch.reshape(tuio, (-1, ))
print(torch.LongTensor(tuio))

tensor([1, 2, 0, 3, 4, 5])
